In [ ]:
import ssl , sys
ssl._create_default_https_context = ssl._create_unverified_context
sys.path.append('./manipulation_library.py') # add the path of manipulation_library.py


In [ ]:
from glob import glob

In [ ]:
from manipulation_library import *


In [ ]:
path_to_folder = "/".join(os.getcwd().split("/")[:-1])

input_coral_image = "data/raw/Platygyra2.jpeg"
path_absolute_to_image = os.path.join(path_to_folder,input_coral_image)



In [ ]:
print(path_absolute_to_image)

In [ ]:
image = get_image(path_absolute_to_image)
width_original = image.shape[1] 
height_original = image.shape[0] 
image = cv2.resize(image, (1800,1200), interpolation = cv2.INTER_AREA)
# list_of_images, titles  = process_images(image = image, masks= masks)


In [ ]:
experiment_grid(model_type="vit_b", image=image)

In [ ]:
# lets make a selection of the masks that are of interest for the user 
mask_of_interest = [0,1,4,5,6,7,9,10,11]

In [ ]:
def multiple_mask_output(model_type, image):
    mask_generator = load_sam_model(model_type=model_type)
    masks = mask_generator.generate(image=image)
    list_of_images , titles , image_dataframe = process_images_and_sort_by_coordinates(image = image, masks= masks)
    return list_of_images, titles, image_dataframe

In [ ]:
def concatenate_images(images, direction='horizontal'):
    """
    Concatenates a list of images either horizontally or vertically.

    Args:
        images (list): List of images to concatenate.
        direction (str): Direction to concatenate images ('horizontal' or 'vertical').

    Returns:
        concatenated_image: The concatenated image.
    """
    if direction not in ['horizontal', 'vertical']:
        raise ValueError("Direction must be 'horizontal' or 'vertical'")

    # Convert images to numpy arrays if they are not already
    images = [np.array(img) for img in images]

    if direction == 'horizontal':
        concatenated_image = np.hstack(images)
    else:
        concatenated_image = np.vstack(images)

    return concatenated_image

In [ ]:
list_of_images, titles, image_dataframe = multiple_mask_output(model_type="vit_b", image=image)
    
  

In [ ]:
image_dataframe.iloc[mask_of_interest]

In [ ]:
type(list_of_images)

In [ ]:
selected_images = [ list_of_images[x] for x in mask_of_interest]

In [ ]:
def add_padding_to_images(images, target_shape):
    """
    Adds padding to a list of images to match the target shape.

    Args:
        images (list): List of numpy.ndarray images to pad.
        target_shape (tuple): Target shape (height, width) to pad images to.

    Returns:
        padded_images: List of padded images.
    """
    padded_images = []
    target_height, target_width = target_shape

    for img in images:
        height, width, _ = img.shape
        top_pad = (target_height - height) // 2
        bottom_pad = target_height - height - top_pad
        left_pad = (target_width - width) // 2
        right_pad = target_width - width - left_pad

        padded_img = cv2.copyMakeBorder(
            img, top_pad, bottom_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, value=[0, 0, 0]
        )
        padded_images.append(padded_img)

    return padded_images

def stack_images(images, direction='horizontal'):
    """
    Stacks a list of numpy.ndarray images either horizontally or vertically.

    Args:
        images (list): List of numpy.ndarray images to stack.
        direction (str): Direction to stack images ('horizontal' or 'vertical').

    Returns:
        stacked_image: The stacked image as a numpy.ndarray.
    """
    if direction not in ['horizontal', 'vertical']:
        raise ValueError("Direction must be 'horizontal' or 'vertical'")

    # Determine the target shape for padding
    max_height = max(img.shape[0] for img in images)
    max_width = max(img.shape[1] for img in images)
    target_shape = (max_height, max_width)

    # Add padding to images to match the target shape
    padded_images = add_padding_to_images(images, target_shape)

    if direction == 'horizontal':
        stacked_image = np.hstack(padded_images)
    else:
        stacked_image = np.vstack(padded_images)

    return stacked_image

In [ ]:
stacked_image = stack_images(selected_images, direction="horizontal")

In [ ]:
plt.imshow(stacked_image)
plt.axis('off')

In [ ]:
def RGB2HEX(color):
    return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

def get_colors(image, number_of_colors, show_chart):
    # Drop all black pixels from the image
    non_black_pixels = image[np.any(image != [0, 0, 0], axis=-1)]
    
    modified_image = non_black_pixels.reshape(non_black_pixels.shape[0], 3)
    
    clf = KMeans(n_clusters=number_of_colors, n_init='auto', random_state=73)
    labels = clf.fit_predict(modified_image)
    
    counts = Counter(labels)
    # Sort to ensure correct color percentage
    counts = dict(sorted(counts.items()))
    
    center_colors = clf.cluster_centers_

    # We get ordered colors by iterating through the keys
    ordered_colors = [center_colors[i] for i in counts.keys()]
    hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts.keys()]
    rgb_colors = [ordered_colors[i] for i in counts.keys()]

    if show_chart:
        plt.figure(figsize=(8, 6))
        plt.pie(counts.values(), labels=rgb_colors, colors=hex_colors)
    
    return rgb_colors

# # Create a copy of the image without black pixels
# image_without_black = image[np.any(image != [0, 0, 0], axis=-1)].reshape(-1, 3)

def drop_black_from_top_colors(top_colors_list):
    min_values = []
    for i in range(len(top_colors_list)):
        curr_color = rgb2lab(np.uint8(np.asarray([[top_colors_list[i]]])))
        diff = deltaE_cie76((0, 0, 0), curr_color)
        # print (diff, type(diff))
        min_values.append(diff[0][0])
        lowest_value_index = np.argmin(min_values) 
    top_colors_list.pop(lowest_value_index)
    return top_colors_list


def match_image_by_color(image, color, threshold = 60, number_of_colors = 10): 
    
    image_colors = get_colors(image, number_of_colors, False)
    # discard black
    image_colors = drop_black_from_top_colors(image_colors)
    selected_color = rgb2lab(np.uint8(np.asarray([[color]])))

    diff_list =[]
    for i in range(len(image_colors)):
        curr_color = rgb2lab(np.uint8(np.asarray([[image_colors[i]]])))
        diff = deltaE_cie76(selected_color, curr_color)
        # print(diff[0][0])
        diff_list.append(diff[0][0])
    diff_avg = np.mean(diff_list)
    if diff_avg < threshold:
        return diff_avg
    else:
        # a euclidian difference of 1000 should be noticible 
        return 1000
    
def calculate_distances_to_colors(image):
    # color chart but in RGB 
    color_map_RGB = {
    'B1': (247, 248, 232),
    'B2': (243, 244, 192),
    'B3': (234, 235, 137),
    'B4': (200, 206, 57),
    'B5': (148, 157, 56),
    'B6': (92, 116, 52),
    'C1': (247, 235, 232),
    'C2': (246, 201, 192),
    'C3': (240, 156, 136),
    'C4': (207, 90, 58),
    'C5': (155, 50, 32),
    'C6': (101, 27, 13),
    'D1': (246, 235, 224),
    'D2': (246, 219, 191),
    'D3': (239, 188, 135),
    'D4': (211, 147, 78),
    'D5': (151, 89, 36),
    'D6': (106, 58, 22),
    'E1': (247, 242, 227),
    'E2': (246, 232, 191),
    'E3': (240, 213, 136),
    'E4': (209, 174, 68),
    'E5': (155, 124, 45),
    'E6': (111, 85, 34)
    }
    
    # get the distance 
    final_distances = {}
    for key in color_map_RGB.keys():
        max_val = match_image_by_color( image=image, color=color_map_RGB[key], number_of_colors=6)
        if max_val != 0 :
            final_distances[key]=max_val
    df_final = pd.DataFrame.from_dict(final_distances,orient='index',columns=["Distance"])
    df_final.sort_values(by="Distance",ascending=True,inplace=True)
    color_keys_selected= df_final.head(n=3).index.to_list()
    color_selected_distance = df_final["Distance"].head(n=3).to_list()
    lower_y_limit = color_selected_distance[0] - 0.5
    higher_y_limit = color_selected_distance[-1] + 0.5
    hex_colors_map = [RGB2HEX(color_map_RGB[key]) for key in color_keys_selected]
    return color_keys_selected, color_selected_distance, lower_y_limit, higher_y_limit,hex_colors_map

def plot_compare(img1_rgb,color_keys_selected, color_selected_distance, lower_y_limit, higher_y_limit,hex_colors_map):
    # Create a figure and subplots
    fig, (ax1, ax2) = plt.subplots(nrows=1,ncols=2,figsize=(20, 10))  # Adjust figsize as needed

    # Display the images
    ax1.imshow(img1_rgb)
    ax1.set_title("Image 1")
    ax1.axis('off') 

    ax2.bar(color_keys_selected,color_selected_distance ,color = hex_colors_map)
    ax2.set_title("Image 2")

    ax2.set_ylabel("Euclidian distance from \n top 5 colors detected")
    ax2.set_xlabel("Color code in chart")
    
    plt.xlabel("Color code in chart")
    plt.ylim(lower_y_limit,higher_y_limit)

    # Adjust spacing between subplots
    plt.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
get_colors(stacked_image, 10, True)

In [ ]:
color_keys_selected, color_selected_distance, lower_y_limit, higher_y_limit,hex_colors_map = calculate_distances_to_colors(stacked_image)

In [ ]:
plot_compare(stacked_image,color_keys_selected, color_selected_distance, lower_y_limit, higher_y_limit,hex_colors_map)